In [1]:
# Import the required libraries and dependencies
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
# Read the CSV file into a Pandas DataFrame
# Set the index using the Ticker column
df_stocks = pd.read_csv(
    Path("Resources/tsx-energy-2018.csv"),                
    index_col="Ticker"
)

# Review the DataFrame
df_stocks.head()

,CompanyName,MeanOpen,MeanHigh,MeanLow,MeanClose,MeanVolume,AnnualReturn,AnnualVariance,EnergyType
Ticker,,,,,,,,,
ARX,ARC Resources Ltd.,13.14,13.34,12.91,13.10,1479913.38,-0.7275,0.3590,Oil
CCO,Cameco Corporation,13.70,13.92,13.50,13.70,1203788.22,0.2014,0.3693,Other Energy
CNQ,Canadian Natural Resources Limited,41.97,42.46,41.46,41.91,3218248.68,-0.3461,0.2947,Oil
CVE,Cenovus Energy Inc.,11.96,12.18,11.75,11.95,4566143.56,-0.3219,0.4500,Oil
CPG,Crescent Point Energy Corp.,8.53,8.67,8.36,8.50,3919414.03,-1.0103,0.4597,Other Energy


In [3]:
# Check the shape of the DatFrame
df_stocks.shape

(24, 9)

In [4]:
# Scale price data, return, and variance values
stock_data_scaled = StandardScaler().fit_transform(
    df_stocks[["MeanOpen", "MeanHigh", "MeanLow", "MeanClose", "MeanVolume", "AnnualReturn", "AnnualVariance"]]
)

In [5]:
# Create a new DataFrame with the scaled data
df_stocks_scaled = pd.DataFrame(
    stock_data_scaled,
    columns=["MeanOpen", "MeanHigh", "MeanLow", "MeanClose", "MeanVolume", "AnnualReturn", "AnnualVariance"]
)

# Copy the tickers names from the original data
df_stocks_scaled["Ticker"] = df_stocks.index

# Set the Ticker column as index
df_stocks_scaled = df_stocks_scaled.set_index("Ticker")

# Display sample data
df_stocks_scaled.head()

,MeanOpen,MeanHigh,MeanLow,MeanClose,MeanVolume,AnnualReturn,AnnualVariance
Ticker,,,,,,,
ARX,-0.916832,-0.917217,-0.918045,-0.918135,-0.152786,-1.332445,0.460854
CCO,-0.880152,-0.879472,-0.879062,-0.878786,-0.379117,1.695742,0.559411
CNQ,0.971524,0.977848,0.968315,0.971255,1.272074,-0.089092,-0.154415
CVE,-0.994122,-0.992707,-0.994689,-0.993553,2.376902,-0.010201,1.331607
CPG,-1.218785,-1.221130,-1.218673,-1.219807,1.846798,-2.254365,1.424424


In [6]:
# Encode the "EnergyType" column to variables to categorize oil versus non-oil firms. 
oil_dummies = pd.get_dummies(df_stocks["EnergyType"]).astype('int')
oil_dummies.head()

,Oil,Other Energy
Ticker,,
ARX,1,0
CCO,0,1
CNQ,1,0
CVE,1,0
CPG,0,1


In [7]:
# Concatenate the "EnergyType" variables with the scaled data DataFrame.
df_stocks_scaled = pd.concat([df_stocks_scaled, oil_dummies], axis=1)

# Display the sample data
df_stocks_scaled.head()

,MeanOpen,MeanHigh,MeanLow,MeanClose,MeanVolume,AnnualReturn,AnnualVariance,Oil,Other Energy
Ticker,,,,,,,,,
ARX,-0.916832,-0.917217,-0.918045,-0.918135,-0.152786,-1.332445,0.460854,1,0
CCO,-0.880152,-0.879472,-0.879062,-0.878786,-0.379117,1.695742,0.559411,0,1
CNQ,0.971524,0.977848,0.968315,0.971255,1.272074,-0.089092,-0.154415,1,0
CVE,-0.994122,-0.992707,-0.994689,-0.993553,2.376902,-0.010201,1.331607,1,0
CPG,-1.218785,-1.221130,-1.218673,-1.219807,1.846798,-2.254365,1.424424,0,1


In [9]:
# Preapare data for modeling with K-means by removing extra column.
df_stocks_scaled = df_stocks_scaled.iloc[:,:-1]
df_stocks_scaled.head()

,MeanOpen,MeanHigh,MeanLow,MeanClose,MeanVolume,AnnualReturn,AnnualVariance,Oil
Ticker,,,,,,,,
ARX,-0.916832,-0.917217,-0.918045,-0.918135,-0.152786,-1.332445,0.460854,1
CCO,-0.880152,-0.879472,-0.879062,-0.878786,-0.379117,1.695742,0.559411,0
CNQ,0.971524,0.977848,0.968315,0.971255,1.272074,-0.089092,-0.154415,1
CVE,-0.994122,-0.992707,-0.994689,-0.993553,2.376902,-0.010201,1.331607,1
CPG,-1.218785,-1.221130,-1.218673,-1.219807,1.846798,-2.254365,1.424424,0


In [10]:
# Initialize the K-Means model with n_clusters=3
model = KMeans(n_clusters=3)

In [11]:
# Fit the model for the df_stocks_scaled DataFrame
model.fit(df_stocks_scaled)

/Users/samfish/opt/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=3)

In [12]:
# Predict the clusters and then create a new DataFrame with the predicted clusters.
# Predict the model segments (clusters)
stock_clusters = model.predict(df_stocks_scaled)

# View the stock segments
print(stock_clusters)

[2 1 0 2 2 0 1 1 1 0 1 1 2 1 1 1 0 1 2 0 0 0 0 2]


In [13]:
# Create a new column in the DataFrame with the predicted clusters
df_stocks_scaled["StockCluster"] = stock_clusters

# Review the DataFrame
df_stocks_scaled.head()

,MeanOpen,MeanHigh,MeanLow,MeanClose,MeanVolume,AnnualReturn,AnnualVariance,Oil,StockCluster
Ticker,,,,,,,,,
ARX,-0.916832,-0.917217,-0.918045,-0.918135,-0.152786,-1.332445,0.460854,1,2
CCO,-0.880152,-0.879472,-0.879062,-0.878786,-0.379117,1.695742,0.559411,0,1
CNQ,0.971524,0.977848,0.968315,0.971255,1.272074,-0.089092,-0.154415,1,0
CVE,-0.994122,-0.992707,-0.994689,-0.993553,2.376902,-0.010201,1.331607,1,2
CPG,-1.218785,-1.221130,-1.218673,-1.219807,1.846798,-2.254365,1.424424,0,2


In [14]:
# Create a scatter plot with x="AnnualVariance' and y="AnnualReturn"
df_stocks_scaled.hvplot.scatter(
    x="MeanVolume",
    y="AnnualReturn",
    by="StockCluster",
    hover_cols = ["Ticker"], 
    title = "Scatter Plot by Stock Segment - k=3"
)

/Users/samfish/opt/anaconda3/lib/python3.9/site-packages/holoviews/core/util.py:1161: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
/Users/samfish/opt/anaconda3/lib/python3.9/site-packages/holoviews/core/util.py:1161: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
/Users/samfish/opt/anaconda3/lib/python3.9/site-packages/holoviews/core/util.py:1161: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
/Users/samfish/opt/anaconda3/lib/python3.9/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always b

:NdOverlay   [StockCluster]
   :Scatter   [MeanVolume]   (AnnualReturn,Ticker)

In [15]:
# Create the PCA model instance and reduce numner of components so n_components=2
pca = PCA(n_components=2)

In [17]:
# Remove initial clustering result from df
df_stocks_scaled = df_stocks_scaled.iloc[:,:-1]
df_stocks_scaled.head()

,MeanOpen,MeanHigh,MeanLow,MeanClose,MeanVolume,AnnualReturn,AnnualVariance,Oil
Ticker,,,,,,,,
ARX,-0.916832,-0.917217,-0.918045,-0.918135,-0.152786,-1.332445,0.460854,1
CCO,-0.880152,-0.879472,-0.879062,-0.878786,-0.379117,1.695742,0.559411,0
CNQ,0.971524,0.977848,0.968315,0.971255,1.272074,-0.089092,-0.154415,1
CVE,-0.994122,-0.992707,-0.994689,-0.993553,2.376902,-0.010201,1.331607,1
CPG,-1.218785,-1.221130,-1.218673,-1.219807,1.846798,-2.254365,1.424424,0


In [18]:
# Fit the df_stocks_scaled data to the PCA
stocks_pca_data = pca.fit_transform(df_stocks_scaled)

# Review the first five rose of the PCA data
# using bracket notation ([0:5])
stocks_pca_data[:5]

array([[-2.01541918,  0.46518931],
       [-1.62885632, -1.40685588],
       [ 1.85394351,  1.39068316],
       [-2.2941301 ,  1.95995804],
       [-3.04963345,  2.50345178]])

In [19]:
# Calculate the explained variance
pca.explained_variance_ratio_

array([0.64467721, 0.1714023 ])

In [20]:
# Create a newDataFrame with the PCA data
df_stocks_pca = pd.DataFrame(stocks_pca_data, columns=["PC1", "PC2"])

# Copy the tickers names from the original data
df_stocks_pca["Ticker"] = df_stocks.index

# Set the Ticker column as index
df_stocks_pca = df_stocks_pca.set_index("Ticker")

# Review the DataFrame
df_stocks_pca.head()

,PC1,PC2
Ticker,,
ARX,-2.015419,0.465189
CCO,-1.628856,-1.406856
CNQ,1.853944,1.390683
CVE,-2.294130,1.959958
CPG,-3.049633,2.503452


In [21]:
# Replicate the K-means algorithm with the PCA data.
# Initialize the K-Means model with n_clusters=3
model = KMeans(n_clusters=3)

# Fit the model for the df_stocks_pca DataFrame
model.fit(df_stocks_pca)

# Predict the model segments (clusters)
stock_clusters = model.predict(df_stocks_pca)

# Print the stock segments
print(stock_clusters)

[1 2 0 1 1 0 2 2 2 0 2 2 1 2 2 2 0 2 1 0 0 0 0 1]


/Users/samfish/opt/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [22]:
# Create a copy of the df_stocks_pca DataFrame and name it as df_stocks_pca_predictions
df_stocks_pca_predictions = df_stocks_pca.copy()

# Create a new column in the DataFrame with the predicted clusters
df_stocks_pca_predictions["StockCluster"] = stock_clusters

# Review the DataFrame
df_stocks_pca_predictions.head()

,PC1,PC2,StockCluster
Ticker,,,
ARX,-2.015419,0.465189,1
CCO,-1.628856,-1.406856,2
CNQ,1.853944,1.390683,0
CVE,-2.294130,1.959958,1
CPG,-3.049633,2.503452,1


In [23]:
# Create the scatter plot with x="PC1" and y="PC2"
df_stocks_pca_predictions.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="StockCluster",
    hover_cols = ["Ticker"], 
    title = "Scatter Plot by Stock Segment - PCA=2"
)

/Users/samfish/opt/anaconda3/lib/python3.9/site-packages/holoviews/core/util.py:1161: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
/Users/samfish/opt/anaconda3/lib/python3.9/site-packages/holoviews/core/util.py:1161: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
/Users/samfish/opt/anaconda3/lib/python3.9/site-packages/holoviews/core/util.py:1161: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
/Users/samfish/opt/anaconda3/lib/python3.9/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always b

:NdOverlay   [StockCluster]
   :Scatter   [PC1]   (PC2,Ticker)

In [24]:
# Combine results with original data
df_all = pd.concat([df_stocks, df_stocks_pca_predictions],axis=1)
df_all

,CompanyName,MeanOpen,MeanHigh,MeanLow,MeanClose,MeanVolume,AnnualReturn,AnnualVariance,EnergyType,PC1,PC2,StockCluster
Ticker,,,,,,,,,,,,
ARX,ARC Resources Ltd.,13.14,13.34,12.91,13.10,1479913.38,-0.7275,0.3590,Oil,-2.015419,0.465189,1
CCO,Cameco Corporation,13.70,13.92,13.50,13.70,1203788.22,0.2014,0.3693,Other Energy,-1.628856,-1.406856,2
CNQ,Canadian Natural Resources Limited,41.97,42.46,41.46,41.91,3218248.68,-0.3461,0.2947,Oil,1.853944,1.390683,0
CVE,Cenovus Energy Inc.,11.96,12.18,11.75,11.95,4566143.56,-0.3219,0.4500,Oil,-2.294130,1.959958,1
CPG,Crescent Point Energy Corp.,8.53,8.67,8.36,8.50,3919414.03,-1.0103,0.4597,Other Energy,-3.049633,2.503452,1
ENB,Enbridge Inc.,43.43,43.79,42.90,43.32,3764995.23,-0.1993,0.2339,Other Energy,2.284147,1.227484,0
ERF,Enerplus Corporation,14.66,14.90,14.43,14.64,1113371.53,-0.2554,0.3666,Oil,-1.669026,-0.522050,2
GEI,Gibson Energy Inc.,18.53,18.73,18.33,18.54,356758.55,-0.0273,0.2531,Other Energy,-0.729686,-1.736663,2
HSE,Husky Energy Inc.,18.89,19.14,18.64,18.87,1322354.12,-0.2899,0.3115,Other Energy,-0.996260,-0.524772,2


In [25]:
for cluster in df_all.StockCluster.unique():
    print(cluster)
    print(df_all[df_all.StockCluster == cluster]["CompanyName"])

1
Ticker
ARX               ARC Resources Ltd.
CVE              Cenovus Energy Inc.
CPG      Crescent Point Energy Corp.
MEG                 MEG Energy Corp.
VII    Seven Generations Energy Ltd.
WCP          Whitecap Resources Inc.
Name: CompanyName, dtype: object
2
Ticker
CCO         Cameco Corporation
ERF       Enerplus Corporation
GEI         Gibson Energy Inc.
HSE          Husky Energy Inc.
IPL        Inter Pipeline Ltd.
KEY               Keyera Corp.
MTL          Mullen Group Ltd.
PXT       Parex Resources Inc.
PKI       Parkland Corporation
PSK    PrairieSky Royalty Ltd.
Name: CompanyName, dtype: object
0
Ticker
CNQ    Canadian Natural Resources Limited
ENB                         Enbridge Inc.
IMO                  Imperial Oil Limited
PPL          Pembina Pipeline Corporation
SU                     Suncor Energy Inc.
TRP                 TC Energy Corporation
TOU                  Tourmaline Oil Corp.
VET                 Vermilion Energy Inc.
Name: CompanyName, dtype: object


In [27]:
pca.components_

array([[ 0.45909865,  0.45901713,  0.45920688,  0.45914138,  0.00921683,
         0.12452821, -0.3752216 ,  0.02159781],
       [ 0.07940306,  0.07958659,  0.078522  ,  0.07863812,  0.78850296,
        -0.51967858,  0.24270737,  0.15589053]])

In [28]:
# Use elbow method to find best value for k with pca data.

# Create a list with the number of k-values to try
# Use a range from 1 to 11
k = list(range(1, 11))

In [29]:
# Create an empy list to store the inertia values
inertia = []

In [30]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_stocks_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(df_stocks_pca)
    inertia.append(model.inertia_)

/Users/samfish/opt/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/samfish/opt/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/samfish/opt/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/samfish/opt/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init

In [31]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {
    "k": k,
    "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(elbow_data_pca)

In [32]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot_pca = df_elbow_pca.hvplot.line(x="k", y="inertia", title="Elbow Curve Using PCA Data", xticks=k)
elbow_plot_pca

/Users/samfish/opt/anaconda3/lib/python3.9/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/Users/samfish/opt/anaconda3/lib/python3.9/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Curve   [k]   (inertia)

# Analysis:
# Based on the the PCA results and the elbow curve mthod we are able to get similar results with fewer features and also see that 3 is still the best value for k. 